This notebook samples arctic dem product and albedo from random sampling points


Users should change the size of spatial window when extracting the pixel values. 

In [1]:
import geemap
import ee
import pandas as pd

# Prepare sample sites

In [2]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
pd.to_datetime(1338681600000, unit="ms")

Timestamp('2012-06-03 00:00:00')

In [4]:
greenlandmask = ee.Image('OSU/GIMP/2000_ICE_OCEAN_MASK') \
                   .select('ice_mask').eq(1) #'ice_mask', 'ocean_mask'
arcticDEM = ee.Image('UMN/PGC/ArcticDEM/V3/2m_mosaic')

arcticDEMgreenland = arcticDEM.updateMask(greenlandmask)

visPara = {'min': 0,  'max': 3000.0, 'palette': ['0d13d8', '60e1ff', 'ffffff']}
# visPara = {'min': 0,  'max': 3000.0, 'palette': palette}

Map.addLayer(arcticDEMgreenland, visPara, 'Arctic DEM terrain')
Map.setCenter(-41.0, 74.0, 3)
# Map.add_colorbar(visPara, label="Elevation (m)", discrete=False, orientation="vertical", layer_name="Arctic DEM terrain")

In [5]:
randomPoints = ee.FeatureCollection("projects/ee-deeppurple/assets/topography/randomPoints300")
Map.addLayer(randomPoints, {}, 'Random points')
# geemap.ee_export_vector(randomPoints, 'randomPoints.kmz')

In [6]:
sampleN = randomPoints.size().getInfo()
sampleList = randomPoints.geometry().coordinates().getInfo()

## Albedo

In [7]:
# https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard by https://github.com/guiattard
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]]#.dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [8]:
date_start = ee.Date.fromYMD(2009, 8, 16)
date_end = ee.Date.fromYMD(2017, 3, 12)
modis = ee.ImageCollection('MODIS/006/MOD10A1').filterDate(date_start, date_end).select("Snow_Albedo_Daily_Tile")

In [9]:
# dem related functions
def demtool(img):
    demproduct = ee.Terrain.products(img)
    return demproduct

In [11]:
for i in range(sampleN):

    print('The feature id is: %d' %i)
    
    
    aoi = ee.Geometry.Point(sampleList[i])
    # Map.addLayer(aoi, {}, stationName)
    
    # arctic dem strip
    arcticdemStrip = ee.ImageCollection('UMN/PGC/ArcticDEM/V3/2m') \
                  .filterBounds(aoi) \
                  .select('elevation') \
                  .sort('system:time_start', True).map(demtool)

    pointValue = modis.getRegion(aoi, 90).getInfo() # The number e.g. 500 is the buffer size
    dfpoint = ee_array_to_df(pointValue, ["Snow_Albedo_Daily_Tile"])
    pointValueFile = 'randomPoints/modis30m/albedo/' + str(i) + '.csv'
    dfpoint.dropna().to_csv(pointValueFile, mode='w', index=False, header=True)
    
    pointValue = arcticdemStrip.getRegion(aoi, 90).getInfo()
    pointEndtime = arcticdemStrip.aggregate_array("system:time_end").getInfo()

    dfpoint = ee_array_to_df(pointValue, ['elevation', 'slope', 'aspect', 'hillshade'])
    dfpoint["time_end"] = pointEndtime
    pointValueFile = 'randomPoints/modis30m/dem/' + str(i) + '.csv'
    dfpoint.dropna().to_csv(pointValueFile, mode='w', index=False, header=True)

The feature id is: 0
The feature id is: 1
The feature id is: 2
The feature id is: 3
The feature id is: 4
The feature id is: 5
The feature id is: 6
The feature id is: 7
The feature id is: 8
The feature id is: 9
The feature id is: 10
The feature id is: 11
The feature id is: 12
The feature id is: 13
The feature id is: 14
The feature id is: 15
The feature id is: 16
The feature id is: 17
The feature id is: 18
The feature id is: 19
The feature id is: 20
The feature id is: 21
The feature id is: 22
The feature id is: 23
The feature id is: 24
The feature id is: 25
The feature id is: 26
The feature id is: 27
The feature id is: 28
The feature id is: 29
The feature id is: 30
The feature id is: 31
The feature id is: 32
The feature id is: 33
The feature id is: 34
The feature id is: 35
The feature id is: 36
The feature id is: 37
The feature id is: 38
The feature id is: 39
The feature id is: 40
The feature id is: 41
The feature id is: 42
The feature id is: 43
The feature id is: 44
The feature id is: 4